# Candle Extraction from Python Binance

Following will demonstrate method to extract candle data from Python Binance Library. 

https://python-binance.readthedocs.io/en/latest/market_data.html#id6
https://developers.binance.com/docs/binance-spot-api-docs/websocket-api/market-data-requests



# 1. You need to have an API Key & Secret. 


In [2]:
import pandas as pd
from binance.client import Client
import time
from datetime import datetime
import pytz
import util

api_key = ""
api_secret = ""
client = Client(api_key, api_secret)

print(client)



# 2. Collect data for all symbols into a dictionary of DataFrames
Merge DataFrames on 'datetime' & Sort by datetime to ensure chronological order


In [10]:
# config
symbols = ['BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'BNBUSDT', 'TRXUSDT', 'DOGEUSDT'] #Pairs Input
interval = Client.KLINE_INTERVAL_1HOUR #Interval
limit = 1000 #Limit 
start_time_str = "2024-01-01 00:00:00" #Start Time
end_time_str = "2025-01-01 00:00:00" # End Time

start_time = util.str_to_timestamp(start_time_str)
end_time = util.str_to_timestamp(end_time_str)

dataframes = {}
for symbol in symbols:
    klines = util.get_kline_time(client, symbol, interval, start_time, end_time, limit)
    symbol_data = [{
        'datetime': util.timestamp_to_datetime(kline[0]),
        symbol: float(kline[4])  # Closing price
    } for kline in klines]
    dataframes[symbol] = pd.DataFrame(symbol_data)

# Merge DataFrames on 'datetime'
df = dataframes[symbols[0]]
for symbol in symbols[1:]:
    df = df.merge(dataframes[symbol], on='datetime', how='outer')

# Sort by datetime to ensure chronological order
df = df.sort_values('datetime').reset_index(drop=True)

print(df)

csv_filename = f'ASSETS_{interval}_binance_candle.csv'
df.to_csv(csv_filename, index=False)


                 datetime   BTCUSDT  ETHUSDT  SOLUSDT  BNBUSDT  TRXUSDT  \
0     2024-01-01 00:00:00  42475.23  2295.51   101.96   314.40  0.10832   
1     2024-01-01 01:00:00  42613.56  2303.72   104.12   315.30  0.10820   
2     2024-01-01 02:00:00  42581.10  2293.02   103.69   310.90  0.10781   
3     2024-01-01 03:00:00  42330.49  2273.81   103.07   309.20  0.10711   
4     2024-01-01 04:00:00  42399.99  2279.55   102.62   309.20  0.10654   
...                   ...       ...      ...      ...      ...      ...   
8780  2024-12-31 20:00:00  93564.04  3346.20   194.02   702.51  0.25510   
8781  2024-12-31 21:00:00  93899.68  3352.75   193.16   702.80  0.25550   
8782  2024-12-31 22:00:00  93488.84  3339.88   191.13   701.07  0.25460   
8783  2024-12-31 23:00:00  93576.00  3337.78   189.31   702.30  0.25480   
8784  2025-01-01 00:00:00  94401.14  3363.70   192.54   709.12  0.25540   

      DOGEUSDT  
0      0.08983  
1      0.09016  
2      0.08980  
3      0.08921  
4      0.08910